<a href="https://colab.research.google.com/github/deborahmasibo/Hamoye/blob/stage_c/Stability_of_the_Grid_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [52]:
# Imports
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score, classification_report
import os
from google.colab import drive

# Loading Data

In [2]:
# Mounting drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Accessing required working directory
os.chdir('/content/drive/My Drive/Hamoye/Stage C/Quiz')
# Loading dataset and parsing dates
grid = pd.read_csv('Data_for_UCI_named.csv')

# Data Understanding

## preview

In [4]:
# 1st 5 rows
grid.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,0.055347,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,-0.005957,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,0.003471,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,0.028871,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,0.049860,unstable


In [5]:
# Last 5 rows
grid.tail()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf
9995,2.930406,9.487627,2.376523,6.187797,3.343416,-0.658054,-1.449106,-1.236256,0.601709,0.779642,0.813512,0.608385,0.023892,unstable
9996,3.392299,1.274827,2.954947,6.894759,4.349512,-1.663661,-0.952437,-1.733414,0.502079,0.567242,0.285880,0.366120,-0.025803,stable
9997,2.364034,2.842030,8.776391,1.008906,4.299976,-1.380719,-0.943884,-1.975373,0.487838,0.986505,0.149286,0.145984,-0.031810,stable
9998,9.631511,3.994398,2.757071,7.821347,2.514755,-0.966330,-0.649915,-0.898510,0.365246,0.587558,0.889118,0.818391,0.037789,unstable
9999,6.530527,6.781790,4.349695,8.673138,3.492807,-1.390285,-1.532193,-0.570329,0.073056,0.505441,0.378761,0.942631,0.045263,unstable


## Information

In [6]:
# Dataset datatypes
grid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   tau1    10000 non-null  float64
 1   tau2    10000 non-null  float64
 2   tau3    10000 non-null  float64
 3   tau4    10000 non-null  float64
 4   p1      10000 non-null  float64
 5   p2      10000 non-null  float64
 6   p3      10000 non-null  float64
 7   p4      10000 non-null  float64
 8   g1      10000 non-null  float64
 9   g2      10000 non-null  float64
 10  g3      10000 non-null  float64
 11  g4      10000 non-null  float64
 12  stab    10000 non-null  float64
 13  stabf   10000 non-null  object 
dtypes: float64(13), object(1)
memory usage: 1.1+ MB


## Description

In [7]:
# Dataset description
grid.describe()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,5.250000,5.250001,5.250004,5.249997,3.750000,-1.250000,-1.250000,-1.250000,0.525000,0.525000,0.525000,0.525000,0.015731
std,2.742548,2.742549,2.742549,2.742556,0.752160,0.433035,0.433035,0.433035,0.274256,0.274255,0.274255,0.274255,0.036919
min,0.500793,0.500141,0.500788,0.500473,1.582590,-1.999891,-1.999945,-1.999926,0.050009,0.050053,0.050054,0.050028,-0.080760
25%,2.874892,2.875140,2.875522,2.874950,3.218300,-1.624901,-1.625025,-1.624960,0.287521,0.287552,0.287514,0.287494,-0.015557
50%,5.250004,5.249981,5.249979,5.249734,3.751025,-1.249966,-1.249974,-1.250007,0.525009,0.525003,0.525015,0.525002,0.017142
75%,7.624690,7.624893,7.624948,7.624838,4.282420,-0.874977,-0.875043,-0.875065,0.762435,0.762490,0.762440,0.762433,0.044878
max,9.999469,9.999837,9.999450,9.999443,5.864418,-0.500108,-0.500072,-0.500025,0.999937,0.999944,0.999982,0.999930,0.109403


# Data Preperation

## Missing Data

In [8]:
# Checking for missing data
grid.isnull().any().any()

False

There are no missing values.

## Duplicates

In [9]:
# Checking for duplicates
grid.duplicated().any().any()

False

There are no duplicates in the data.

## Outliers

In [10]:
# Checking for outliers

# Outliers function
def outliers(data):
  # IQR
  Q1, Q3, IQR = 0, 0, 0
  outliers = pd.DataFrame()
  # Numerical columns
  numerical = data.select_dtypes(include = ['int64', 'float64'])
  Q1 = numerical.quantile(0.25)
  Q3 = numerical.quantile(0.75)
  IQR = Q3 - Q1
  # Outliers
  outliers = numerical[((numerical < (Q1 - 1.5 * IQR)) |(numerical > (Q3 + 1.5 * IQR))).any(axis=1)]
  print(f'Number of outliers = {outliers.shape[0]}')
  print(f'Percentage = {(outliers.shape[0]/data.shape[0])*100}%')

In [11]:
# Function call
outliers(grid)

Number of outliers = 1
Percentage = 0.01%


There is only 1 outlier present.

# Tag Along

Because of the direct relationship between 'stab' and 'stabf' ('stabf' = 'stable' if 'stab' <= 0, 'unstable' otherwise), 'stab' should be dropped and 'stabf' will remain as the sole dependent variable (binary classification).

Split the data into an 80-20 train-test split with a random state of “1”. Use the standard scaler to transform the train set (x_train, y_train) and the test set (x_test). Use scikit learn to train a random forest and extra trees classifier. And use xgboost and lightgbm to train an extreme boosting model and a light gradient boosting model. Use random_state = 1 for training all models and evaluate on the test set. Answer the following questions:

## Dropping the Stab Column

In [12]:
# Dropping 'stab'
grid.drop('stab', axis = 1, inplace = True)
# Confirming change
grid.columns

Index(['tau1', 'tau2', 'tau3', 'tau4', 'p1', 'p2', 'p3', 'p4', 'g1', 'g2',
       'g3', 'g4', 'stabf'],
      dtype='object')

# Label Encoding

In [13]:
# Encoding
encoder = LabelEncoder()
grid.stabf = encoder.fit_transform(grid['stabf'])
# Cheching changes
grid.stabf.head()

0    1
1    0
2    1
3    1
4    1
Name: stabf, dtype: int64

### Label distribution

In [14]:
# Checking for imbalance
grid.stabf.value_counts()

1    6380
0    3620
Name: stabf, dtype: int64

There is a class imbalance.

## Train-test-split

In [15]:
# Features and label
X = grid.drop('stabf', axis = 1).values
y = grid.stabf.values.reshape(-1, 1)

# Train-test-split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, 
                                                    random_state = 1)

## SMOTE

In [36]:
# Over sampling
# smote = SMOTE(random_state=1)
# X_train_balanced, y_balanced = smote.fit_resample(X_train, y_train)

## Standard Scaler

In [44]:
# Standard scaling
scaler = StandardScaler()
train_scaled = scaler.fit_transform(np.append(X_train, y_train, axis = 1))
X_train_scaled, y_train_scaled = train_scaled[:, :-1], train_scaled[:, -1]
test_scaled = scaler.transform(np.append(X_test, y_test, axis = 1))
X_test_scaled, y_test_scaled = test_scaled[:, :-1], test_scaled[:, -1]

#### Performance Metrics

In [45]:
# Model performance metrics function
def performance_metrics(y_test, y_pred):
  # Accuracy
  acc = accuracy_score(y_test, y_pred)
  # Precision
  prec = precision_score(y_test, y_pred)
  # Recall
  rec = recall_score(y_test, y_pred)
  # F1 Score
  f1 = f1_score(y_test, y_pred)
  
  # Results
  print(f'Accuracy = {round(acc, 4)}')
  print(f'Precision = {round(prec, 4)}')
  print(f'Recall = {round(rec, 4)}')
  print(f'F1 = {round(f1, 4)}')


# Q 14: Random Forest

In [46]:
# Training
rf = RandomForestClassifier(random_state = 1)
rf.fit(X_train_scaled, y_train.reshape(y_train.shape[0]))
# Prediction
rf_preds = rf.predict(X_test_scaled)
# Performance
performance_metrics(y_test, rf_preds)

Accuracy = 0.929
Precision = 0.9341
Recall = 0.9573
F1 = 0.9456


# Q 15: XGBoost

In [ ]:
# Training
xbg = XGBClassifier(random_state = 1)
xbg.fit(X_train_scaled, y_train)
# Prediction
xgb_preds = xbg.predict(X_test_scaled)
# Performance
performance_metrics(y_test, xgb_preds)

Accuracy = 0.9455
Precision = 0.951
Recall = 0.9651
F1 = 0.958


# Q 16: Lightgbm

In [48]:
# Training
lgbm = LGBMClassifier()
lgbm.fit(X_train_scaled, y_train.reshape(y_train.shape[0]))
# Prediction
lgbm_preds = lgbm.predict(X_test_scaled)
# Performance
performance_metrics(y_test, lgbm_preds)

Accuracy = 0.9375
Precision = 0.9415
Recall = 0.9627
F1 = 0.952


# Q 17: RandomizedSearchCV ExtraTreesClassifier

In [60]:
et = ExtraTreesClassifier(random_state = 1)
params = {'n_estimators': [100, 300, 500, 1000],
          'min_samples_split': [2, 5, 7],
          'min_samples_leaf': [4, 6, 8],
          'max_features': ['auto', 'log2', None]
}
clf = RandomizedSearchCV(et, params, cv = 5, n_iter = 10, scoring = 'accuracy', n_jobs = -1, verbose = 1 , random_state = 1)

In [61]:
clf.fit(X_train_scaled, y_train.reshape(y_train.shape[0]))

Fitting 5 folds for each of 10 candidates, totalling 50 fits


RandomizedSearchCV(cv=5, estimator=ExtraTreesClassifier(random_state=1),
                   n_jobs=-1,
                   param_distributions={'max_features': ['auto', 'log2', None],
                                        'min_samples_leaf': [4, 6, 8],
                                        'min_samples_split': [2, 5, 7],
                                        'n_estimators': [100, 300, 500, 1000]},
                   random_state=1, scoring='accuracy', verbose=1)

In [62]:
clf.best_params_

{'n_estimators': 500,
 'min_samples_split': 5,
 'min_samples_leaf': 6,
 'max_features': None}

In [68]:
c = ExtraTreesClassifier(random_state = 1, n_estimators = 300, min_samples_split = 5, min_samples_leaf = 6, max_features = 'auto')
c.fit(X_train_scaled, y_train.reshape(y_train.shape[0]))
# Prediction
c_preds = c.predict(X_test_scaled)
# Performance
performance_metrics(y_test, c_preds)

/usr/local/lib/python3.8/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


Accuracy = 0.921
Precision = 0.8979
Recall = 0.9899
F1 = 0.9417


In [70]:
c = ExtraTreesClassifier(random_state = 1, n_estimators = 500, min_samples_split = 2, min_samples_leaf = 2, max_features = 'log2')
c.fit(X_train_scaled, y_train.reshape(y_train.shape[0]))
# Prediction
c_preds = c.predict(X_test_scaled)
# Performance
performance_metrics(y_test, c_preds)

Accuracy = 0.932
Precision = 0.9144
Recall = 0.9868
F1 = 0.9492


In [66]:
c = ExtraTreesClassifier(random_state = 1, n_estimators = 100, min_samples_split = 7, min_samples_leaf = 4, max_features = None)
c.fit(X_train_scaled, y_train.reshape(y_train.shape[0]))
# Prediction
c_preds = c.predict(X_test_scaled)
# Performance
performance_metrics(y_test, c_preds)

Accuracy = 0.9295
Precision = 0.9322
Recall = 0.9604
F1 = 0.9461


In [69]:
c = ExtraTreesClassifier(random_state = 1, n_estimators = 1000, min_samples_split = 2, min_samples_leaf = 8, max_features = None)
c.fit(X_train_scaled, y_train.reshape(y_train.shape[0]))
# Prediction
c_preds = c.predict(X_test_scaled)
# Performance
performance_metrics(y_test, c_preds)

Accuracy = 0.927
Precision = 0.93
Recall = 0.9589
F1 = 0.9442


# Q 18: Optimized ExtraTreesClassifier

## Original ExtraTreeClassifer

In [51]:
# Training
et = ExtraTreesClassifier(random_state = 1)
et.fit(X_train_scaled, y_train.reshape(y_train.shape[0]))
# Prediction
et_preds = et.predict(X_test_scaled)
# Performance
performance_metrics(y_test, et_preds)

Accuracy = 0.928
Precision = 0.9218
Recall = 0.9705
F1 = 0.9455


## Optimized

In [ ]:
c = ExtraTreesClassifier(random_state = 1, n_estimators = 500, min_samples_split = 2, min_samples_leaf = 2, max_features = 'log2')
c.fit(X_train_scaled, y_train.reshape(y_train.shape[0]))
# Prediction
c_preds = c.predict(X_test_scaled)
# Performance
performance_metrics(y_test, c_preds)

Accuracy = 0.932
Precision = 0.9144
Recall = 0.9868
F1 = 0.9492


# Q 20: Feature Importance ExtraTreesClassifier

In [71]:
# Feature importance
(c.feature_importances_)

array([0.1256194 , 0.12789499, 0.12249314, 0.12400683, 0.02578867,
       0.02940308, 0.02915945, 0.02912856, 0.09061057, 0.09757471,
       0.10066386, 0.09765673])

In [74]:
# Results
a = pd.DataFrame({'cols' :list(grid.iloc[:, :-1].columns), 
              'vals': list(c.feature_importances_)})

In [79]:
a

,cols,vals
0,tau1,0.125619
1,tau2,0.127895
2,tau3,0.122493
3,tau4,0.124007
4,p1,0.025789
5,p2,0.029403
6,p3,0.029159
7,p4,0.029129
8,g1,0.090611
9,g2,0.097575
